In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# merge_iot_occupancy_into_semantic_minio.py — ajoute IoT & occupancy → TTL semantic + upload MinIO

import os, sys, argparse, json
from pathlib import Path
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS, XSD

# ==== Namespaces ====
BRICK = Namespace("https://brickschema.org/schema/1.1/Brick#")
BF    = Namespace("https://brickschema.org/schema/BrickFrame#")
QUDT  = Namespace("http://qudt.org/schema/qudt/")
UNIT  = Namespace("http://qudt.org/vocab/unit/")
EX    = Namespace("http://example.org/training#")

# ==== MinIO ====
def s3_client(endpoint, access, secret, secure):
    import boto3
    from botocore.config import Config
    return boto3.client("s3",
        endpoint_url=endpoint, aws_access_key_id=access, aws_secret_access_key=secret,
        use_ssl=bool(secure), verify=bool(secure),
        region_name="us-east-1", config=Config(signature_version="s3v4"))

def ensure_bucket(s3, bucket):
    import botocore
    try: s3.head_bucket(Bucket=bucket)
    except botocore.exceptions.ClientError: s3.create_bucket(Bucket=bucket)

def s3_upload(s3, bucket, p:Path, key:str):
    s3.upload_file(str(p), bucket, key)

def add(g,s,p,o): g.add((s,p,o))
def label(g,s,t): add(g,s,RDFS.label,Literal(t))

def first_or(g,tp,fallback):
    x = list(g.subjects(RDF.type,tp))
    return x[0] if x else fallback

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--month", type=str, default="2025-03")
    ap.add_argument("--in_sem_ttl", type=str, default="~/DTE/jne_project/semantic/{month}/training_room_semantic_bms.ttl")
    # chemins refined IoT & occupancy
    ap.add_argument("--iot_ref_csv_uri", type=str, default="minio://refined/sensors/{month}/zone_101_sensors_refined.csv")
    ap.add_argument("--occ_ref_csv_uri", type=str, default="minio://refined/occupancy/{month}/occupancy_refined.csv")
    ap.add_argument("--sem_base", type=str, default="~/DTE/jne_project/semantic")
    # MinIO semantic
    ap.add_argument("--endpoint", type=str, default=os.environ.get("MINIO_ENDPOINT","http://192.168.0.173:9000"))
    ap.add_argument("--access",   type=str, default=os.environ.get("MINIO_ROOT_USER","minioadmin"))
    ap.add_argument("--secret",   type=str, default=os.environ.get("MINIO_ROOT_PASSWORD","minioadmin"))
    ap.add_argument("--bucket",   type=str, default="semantic")
    ap.add_argument("--prefix",   type=str, default="jne_project/semantic")
    ap.add_argument("--secure",   action="store_true")
    ap.add_argument("--no-upload", action="store_true")
    args,_ = ap.parse_known_args()

    month = args.month
    in_ttl = Path(args.in_sem_ttl.format(month=month)).expanduser().resolve()
    sem_dir  = Path(args.sem_base).expanduser().resolve()/month
    meta_dir = Path(args.sem_base).expanduser().resolve()/"meta"/month
    sem_dir.mkdir(parents=True, exist_ok=True); meta_dir.mkdir(parents=True, exist_ok=True)

    # Charger le TTL existant (déjà BIM+Weather+BMS)
    g = Graph()
    for pfx,ns in [("brick",BRICK),("bf",BF),("qudt",QUDT),("unit",UNIT),("ex",EX),("rdfs",RDFS)]:
        g.bind(pfx, ns)
    g.parse(str(in_ttl), format="turtle")

    # ressources
    room = first_or(g, BRICK.Room, EX["Room_101"])
    hvac_sys = first_or(g, BRICK.HVAC_System, EX["HVAC_System"])
    light_sys = first_or(g, BRICK.Lighting_System, EX["Lighting_System"])

    # IoT sensors (TS du CSV refined sensors)
    iot_csv = args.iot_ref_csv_uri.format(month=month)
    iot_points = [
        ("Temp_Sensor",     BRICK.Temperature_Sensor,        UNIT.DEG_C,   "temp_c",    hvac_sys),
        ("Humidity_Sensor", BRICK.Relative_Humidity_Sensor,  UNIT.PERCENT, "humidity",  hvac_sys),
        ("CO2_Sensor",      BRICK.CO2_Sensor,                UNIT.PPM,     "co2",       hvac_sys),
    ]
    for name, klass, unit_uri, col, sys_node in iot_points:
        s = EX[name]
        add(g, s, RDF.type, klass); label(g, s, name.replace("_"," "))
        add(g, s, BRICK.hasLocation, room)
        add(g, sys_node, BRICK.hasPoint, s)
        add(g, s, QUDT.unit, unit_uri)
        add(g, s, BF.hasTimeseriesId, Literal(f"{iot_csv}::{col}", datatype=XSD.string))

    # Occupancy
    occ_csv = args.occ_ref_csv_uri.format(month=month)
    occ_sensor = EX["Occupancy_Sensor"]
    add(g, occ_sensor, RDF.type, BRICK.Occupancy_Sensor); label(g, occ_sensor, "Occupancy Sensor")
    add(g, occ_sensor, BRICK.hasLocation, room)
    add(g, light_sys, BRICK.hasPoint, occ_sensor)  # aussi lié à l'éclairage
    add(g, occ_sensor, QUDT.unit, UNIT.UNITLESS)
    add(g, occ_sensor, BF.hasTimeseriesId, Literal(f"{occ_csv}::presence", datatype=XSD.string))

    occ_level = EX["Occupancy_Level"]
    add(g, occ_level, RDF.type, BRICK.Level_Sensor); label(g, occ_level, "Occupancy Level")
    add(g, occ_level, BRICK.hasLocation, room)
    add(g, occ_level, QUDT.unit, UNIT.UNITLESS)
    add(g, occ_level, BF.hasTimeseriesId, Literal(f"{occ_csv}::level", datatype=XSD.string))

    # Sauvegarde
    out_ttl = sem_dir/"training_room_semantic_full.ttl"
    g.serialize(destination=str(out_ttl), format="turtle")

    manifest = {
        "version":"1.2","month":month,
        "input_semantic_bms": str(in_ttl),
        "iot_csv_refined": iot_csv,
        "occupancy_csv_refined": occ_csv,
        "output_ttl": str(out_ttl),
        "ts_id_pattern":"<csv_uri>::<column>"
    }
    man_path = meta_dir/"semantic_manifest_full.json"
    Path(man_path).write_text(json.dumps(manifest, indent=2), encoding="utf-8")

    # upload
    if not args.no_upload:
        try:
            s3 = s3_client(args.endpoint, args.access, args.secret, args.secure)
            ensure_bucket(s3, args.bucket)
            root = args.prefix.strip("/")
            s3_upload(s3, args.bucket, out_ttl, f"{root}/{month}/training_room_semantic_full.ttl")
            s3_upload(s3, args.bucket, Path(man_path), f"{root}/meta/{month}/semantic_manifest_full.json")
            print("minio:", f"s3://{args.bucket}/{root}/{month}/training_room_semantic_full.ttl")
        except Exception as e:
            print("ERREUR MinIO:", e, file=sys.stderr); sys.exit(3)

    print("local:", out_ttl, "|", man_path)

if __name__ == "__main__":
    main()


minio: s3://semantic/jne_project/semantic/2025-03/training_room_semantic_full.ttl
local: /home/amina/DTE/jne_project/semantic/2025-03/training_room_semantic_full.ttl | /home/amina/DTE/jne_project/semantic/meta/2025-03/semantic_manifest_full.json
